In [3]:
import pandas as pd
import numpy as np
import os

HOME_PATH = os.path.expanduser('~') + '/Projects/ssmsi/'
DATA_PATH = HOME_PATH + 'pickles/results/'
AUX_DATA_PATH = HOME_PATH + 'pickles/corpora/a-to-h/'

document_ids_map = pd.read_pickle(AUX_DATA_PATH + 'a-to-h_id-map.pickle').to_dict()
inverted_document_ids_map = {v: int(k) for k, v in document_ids_map.iteritems()}
coordinates_mapping = pd.read_pickle(AUX_DATA_PATH + 'a-to-h_coord-map.pickle').to_dict()

### Option: Rogers LDA

In [2]:
thetas = pd.read_pickle(DATA_PATH + 'lda_rogers/rogers_thetas.pickle')

### Option: Gibbs LDA

In [4]:
thetas_all = pd.read_pickle(DATA_PATH + 'lda_gibbs/gibbs_thetas.pickle')
thetas = thetas_all[-1,:,:]

### Mapping the documents to the coordinates


In [19]:
NUMBER_OF_COLUMNS = 776
NUMBER_OF_ROWS = 8

TOPIC_ID = 7

# Mapping to the z values.
z_matrix = np.zeros(shape=(NUMBER_OF_ROWS, NUMBER_OF_COLUMNS))
for i, doc in enumerate(thetas):
    doc_id = i  
#     doc_id = inverted_document_ids_map[i] #TOGGLE FOR ROGERS LDA 
    if doc_id in coordinates_mapping:
        doc_coord = coordinates_mapping[doc_id]
        z_matrix[doc_coord['y']][doc_coord['x']] = doc[TOPIC_ID]

# Creating the x and y axes.         
x_vector = np.zeros(shape=(NUMBER_OF_COLUMNS, 1))
for i in range(0, NUMBER_OF_COLUMNS):
    x_vector[i] = i
y_vector = np.zeros(shape=(NUMBER_OF_ROWS, 1))
for i in range(0, NUMBER_OF_ROWS):
    y_vector[i] = i

In [20]:
import plotly.plotly as py
import plotly.graph_objs as go

layout = go.Layout(title='\'Topic %s\' probability distribution.' % TOPIC_ID,
                   height=350,
                   xaxis=dict(title='x'),
                   yaxis=dict(title='y'))
data = [go.Heatmap(z=z_matrix,
                   x=x_vector,
                   y=y_vector,
                   colorscale='Electric',
                   colorbar=dict(title='Likelihood'))]
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='topic_%s' % TOPIC_ID)